In [507]:
import openai
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv2D, Flatten, Dropout, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.optimizers import Adam, AdamW
from keras.metrics import RootMeanSquaredError
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


In [15]:
data_x = np.load('./raw_data/train/y_smp_train.npy')
data_y = np.load('./raw_data/train/pars_smp_train.npy')

In [16]:
data_x.shape, data_y.shape

((1000000, 200, 3), (1000000, 15, 1))

In [ ]:
time_steps = 200
num_features = 3

In [685]:
from tensorflow.keras import backend as K

In [691]:
def quantile_loss_1(y_true, y_pred):
    # Вычисляем абсолютную ошибку между истинными метками и предсказаниями
    error = y_true - y_pred
    
    # Вычисляем потерю для данного квантиля
    loss = K.mean(K.maximum(0.1 * error, (0.1 - 1) * error))
    return loss

In [695]:
from keras.losses import Huber

In [709]:
def QuantileLoss(perc, delta=1e-4):
    perc = np.array(perc).reshape(-1)
    perc.sort()
    perc = perc.reshape(1, -1)
    def _qloss(y, pred):
        I = tf.cast(y <= pred, tf.float32)
        d = K.abs(y - pred)
        correction = I * (1 - perc) + (1 - I) * perc
        # huber loss
        # huber = Huber()
        huber_loss = Huber(y, pred, delta=delta)
        # huber_loss = K.sum(correction * tf.where(d <= delta, 0.5 * d ** 2 / delta, d - 0.5 * delta), -1)
        # order loss
        q_order_loss = K.sum(K.maximum(0.0, pred[:, :-1] - pred[:, 1:] + 1e-6), -1)
        return huber_loss + q_order_loss
    return _qloss

In [701]:
def combined_loss(quantile, delta=1e-4):
    def loss(y_true, y_pred):
        I = tf.cast(y_true <= y_pred, tf.float32)
        d = tf.abs(y_true - y_pred)
        correction = I * (1 - quantile) + (1 - I) * quantile
        
        # Huber loss
        huber_loss = tf.reduce_sum(correction * tf.where(d <= delta, 0.5 * d ** 2 / delta, d - 0.5 * delta), axis=-1)
        
        # Quantile loss
        quantile_loss = tf.reduce_sum(correction * d, axis=-1)
        
        # Combine the losses
        combined = huber_loss + quantile_loss
        
        return combined
    
    return loss

In [689]:
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

In [708]:
perc_points = [0.1, 0.25, 0.5, 0.75, 0.9]

In [704]:
perc = [0.01, 0.25, 0.5, 0.75, 0.99]
perc = np.array(perc).reshape(-1)
perc.sort()
perc = perc.reshape(1, -1)

In [706]:
perc.shape

(1, 5)

In [710]:
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', input_shape=(200, 3, 1), padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (12,6), activation='relu', strides=(6, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', strides=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(15))

opt = AdamW(learning_rate=0.0001)
model.compile(loss=combined_loss(0.1), optimizer=opt, metrics=[RootMeanSquaredError()])

In [711]:
model.summary()

Model: "sequential_235"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_671 (Conv2D)         (None, 200, 3, 16)        160       
                                                                 
 up_sampling2d_271 (UpSampl  (None, 400, 6, 16)        0         
 ing2D)                                                          
                                                                 
 conv2d_672 (Conv2D)         (None, 400, 6, 32)        4640      
                                                                 
 up_sampling2d_272 (UpSampl  (None, 800, 12, 32)       0         
 ing2D)                                                          
                                                                 
 conv2d_673 (Conv2D)         (None, 132, 7, 32)        73760     
                                                                 
 max_pooling2d_336 (MaxPool  (None, 66, 4, 32)      

In [712]:
X_train = data_x[:10000]
y_train = data_y[:10000]
X_val = data_x[10000:12000]
y_val = data_y[10000:12000]
X_test = data_x[200000:300000]
y_test = data_y[200000:30000]

In [713]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((10000, 200, 3), (10000, 15, 1), (2000, 200, 3), (2000, 15, 1))

In [714]:
model.fit(X_train, y_train,
                epochs=2,
                validation_data=(X_val, y_val), workers=9)

Epoch 1/2
313/313 [==============================] - 67s 211ms/step - loss: 5.2260 - root_mean_squared_error: 1.4040 - val_loss: 2.9598 - val_root_mean_squared_error: 1.3477
Epoch 2/2
313/313 [==============================] - 71s 226ms/step - loss: 3.4182 - root_mean_squared_error: 1.1880 - val_loss: 2.4659 - val_root_mean_squared_error: 1.3029


In [668]:
model.save("Model_test.h5")

c:\Users\wilte\OneDrive\Рабочий стол\projects\sochi-2023\venv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [669]:
from keras.models import load_model

In [671]:
model_new = load_model("Model_test.h5")

In [673]:
model_new.predict(X_train[99:100])

1/1 [==============================] - 0s 103ms/step


array([[1.7580435 , 1.9172012 , 0.44049358, 0.47736034, 1.500695  ,
        0.18440059, 0.9878392 , 2.571119  , 0.4447378 , 0.3627519 ,
        0.5743027 , 0.29489222, 0.03618997, 0.06648906, 0.1102337 ]],
      dtype=float32)

In [ ]:
pred = model.predict(X_train[99:100])

1/1 [==============================] - 0s 67ms/step


In [351]:
pred.shape

(1, 15)

In [279]:
a = np.array(pred)

In [326]:
random_sub = np.load('./raw_data/random_submit.npy')

In [331]:
random_sub.shape

(100000, 15, 6)

In [328]:
a = pd.DataFrame(random_sub[0])

In [358]:
sub1 = np.load('./submission/1.npy')

In [359]:
sub1.shape

(100000, 15, 6)

In [329]:
a

,0,1,2,3,4,5
0,1.740579,0.189885,-0.245752,-0.352857,-0.432121,-0.687127
1,0.498018,1.753163,-1.582503,0.555470,-2.952694,1.254703
2,-1.188928,0.354142,1.682241,-0.209834,0.178970,-0.262190
3,-0.594227,-1.411590,-1.275111,-0.639211,-0.176287,0.648948
4,0.135326,-0.576863,0.199233,0.358695,-1.045038,0.067812
5,-1.132443,-0.920505,-0.425666,1.933943,-0.425191,0.368368
6,-0.373851,-0.178573,-0.642971,1.587044,-1.431810,0.386378
7,-0.243564,-0.470445,1.554300,-1.710335,-0.056500,0.482639
8,-0.222726,-0.090573,0.188107,-0.142399,0.355254,-0.484876
9,-0.728160,-0.880158,-0.492981,0.256155,0.370332,0.202764


In [330]:
import pandas as pd

In [ ]:
np.save('test_sub', a)